In [ ]:
# We import the neccessary packages in the beginning
import os
import pm4py
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.conversion.bpmn import converter as bpmn_converter
from sklearn import tree
import numpy as np
import pandas as pd
import pickle
from pm4py.util import exec_utils
from enum import Enum
from pm4py.algo.discovery.footprints import algorithm as footprints_discovery
from pm4py.visualization.petri_net import visualizer as pn_viz
from pm4py.objects.process_tree.utils import generic as pt_util
from pm4py.objects.process_tree.utils.generic import tree_sort
from pm4py.util.variants_util import get_variant_from_trace
from pm4py.statistics.variants.log.get import get_variants_sorted_by_count
import docplex
import time
from docplex.mp.model import Model
from collections import Iterable
class Parameters(Enum):
    DEBUG = "debug"
    FOLD = "fold"

In [ ]:
# Returns a path to the file selected by the user
# Input: The folder in which to look for the files - the default is the current folder
def ask_for_path(rel_path='', index = -1):
    #Crawl all files in the input folder
    print("The following files are available in the input folder:\n")

    count = 0
    file_list = os.listdir(os.getcwd() + rel_path)
    for file in file_list:
        print(str(count) + " - " + file)
        count+=1

    if(index == -1):
        #Ask for which of the files shall be transformed and select it.
        inp = input("Please choose from the list above which of the files shall be transformed by typing the corresponding number.")
    else:
        #Automatic iteration
        print('Automatic Iteration.')
        inp = index

    input_file = file_list[int(inp)]

    return (os.getcwd() + rel_path + input_file)

In [ ]:
# this function converts a selected file in the path that is the input into a log
def transform_to_log(file_path):
    filename, file_extension = os.path.splitext(file_path)
    x,z =os.path.split(file_path)
    
    if file_extension == '.csv':
        log_csv = pd.read_csv(file,sep=None,encoding='utf-8-sig')
        if z =='mobis_challenge_log_2019.csv' or z =='mobis_challenge_log_2019_only_complete_cases.csv':
            log_csv['end'] = pd.to_datetime(log_csv['end'])
            log_csv['start'] = pd.to_datetime(log_csv['start'])
            log_csv['cost'] = log_csv['cost'].apply(pd.to_numeric, errors='coerce')
            log_csv.rename(columns={'cost': 'case:cost','case':'case:concept:name','activity':'concept:name','end':'time:timestamp', 'user':'org:resource'}, inplace=True)
        log = log_converter.apply(log_csv)
    elif file_extension == '.xes':
        log = pm4py.read_xes(file_path)
        log = pm4py.convert_to_event_log(log)
    elif file_extension == '.dfg':
        log = pm4py.read_dfg(file_path)
    else:
        print("Current filetype is equal to {}. \nPlease input a file with any of the following extensions: - csv; - xes; - dfg".format(str(file_extension)))
        return -1

    return log

In [ ]:
##########
"""Settings"""
##########
# set the input and output path according to the files you want to select
REL_INPUT_PATH = "/Hosseinpour_Jans/" # here lie the event logs (.csv), the to-be model (.bpmn) and the already aligned traces (.pkl)


In [ ]:
file= ask_for_path(REL_INPUT_PATH,6) # adjust to your path
log=transform_to_log(file)
log

In [ ]:
file= ask_for_path(REL_INPUT_PATH,4)# adjust to your path
bpmn_graph = pm4py.read_bpmn(file)
net, initial_marking, final_marking = bpmn_converter.apply(bpmn_graph)
pm4py.view_petri_net(net, initial_marking, final_marking)

In [ ]:
# these functions flatten process trees, which is necessary to handle the set Conc and Ex
def leafs(tree):
    activities = []
    if not len(tree._children) > 0:
        return None
    for child in range(len(tree._children)):
        if len(tree._children[child]._children) == 0:
            activities.append(tree._children[child]) # leafs found
        else:
            activities.append(leafs(tree._children[child])) # next operator found
    return list(flatten(activities))

def flatten(lis):
    for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
        else:
            yield item

def relations(tree, parallels, exclusives):
    if not len(tree._children) > 0:
        return parallels, exclusives

    tree_operator = tree.operator
    if str(tree_operator) == '+' and not 'tau' in str(leafs(tree)):
        parallels[' and '.join(list(str(tree._children[q]) for q in range(len(tree._children))))] = leafs(tree)

    if str(tree_operator) == 'X' and not 'tau' in str(leafs(tree)):
        exclusives[' and '.join(list(str(tree._children[q]) for q in range(len(tree._children))))] = leafs(tree)

    if len(tree._children)>0:
        for child in range(len(tree._children)):
            parallels, exclusives=relations(tree._children[child],parallels, exclusives)
    return parallels, exclusives

In [ ]:
# this funcition generates trace alignments with the adjusted cost function
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    i = 0
    dev = []
    for trace in log:
        no_moves = len(aligned_traces[i]['alignment'])
        for j in range(0, len(aligned_traces[i]['alignment'])):
            if aligned_traces[i]['alignment'][j][1] == None or aligned_traces[i]['alignment'][j][0] == \
                    aligned_traces[i]['alignment'][j][1]:
                next
            else:
                if not str(aligned_traces[i]['alignment'][j]) in dev:
                    dev.append(str(aligned_traces[i]['alignment'][j]))
                #trace.attributes[str(aligned_traces[i]['alignment'][j])]=1
        i += 1

    f = open('aligned_traces_adjusted_large.pkl', 'wb')
    pickle.dump(aligned_traces, f)
    f.close()
    return aligned_traces

In [ ]:
def generate_alignments_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star)



    return aligned_traces

In [ ]:
def discover_patterns(log, net, initial_marking, final_marking, bigM=1000, cost_swap=1, cost_replace=1.1, cost_repeat=1.2):
    xt, z = os.path.split(file)
    time_start = time.clock()
    aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)
    #### get information whether deviation happened after prefix length in DF
    max_moves = 0
    dev = []  # stores all deviations that happened
    for i, trace in enumerate(log):
        no_moves = len(aligned_traces[i]['alignment'])
        if max_moves < no_moves:
            max_moves = no_moves
        for j in range(0, len(aligned_traces[i]['alignment'])):
            if aligned_traces[i]['alignment'][j][1] == None or aligned_traces[i]['alignment'][j][0] == \
                    aligned_traces[i]['alignment'][j][1]:
                next  # we do not care for simultaneous or silent moves
            else:
                if not str(aligned_traces[i]['alignment'][j]) in dev:
                    dev.append(str(aligned_traces[i]['alignment'][j]))

    collect_traces = pd.DataFrame(data=0, columns=dev, index=range(
        len(log)))  # Data Frame that stores the information whether a deviation happened for each trace on trace level
    collect_traces['variant_idx'] = None

    # select only one trace per variant
    variants = pm4py.get_variants(log)
    variant_list = list(variants.keys())
    variant_list = list(','.join(variant_list[e]) for e in range(len(variant_list)))

    for i, trace in enumerate(log):
        collect_traces['variant_idx'][i] = variant_list.index(','.join(list(get_variant_from_trace(log[i]))))
        for j in range(0, len(aligned_traces[i]['alignment'])):
            if aligned_traces[i]['alignment'][j][1] == None or aligned_traces[i]['alignment'][j][0] == \
                    aligned_traces[i]['alignment'][j][1]:
                next
            else:
                collect_traces[str(aligned_traces[i]['alignment'][j])][i] = 1

    var_counts = dict(get_variants_sorted_by_count(variants))
    variants_deviating = pd.DataFrame(data=None, columns=['deviating', 'first_idx', 'count', 'label'], index=variant_list)
    for i in range(len(log)):
        if not pd.isna(variants_deviating['first_idx'][','.join(list(get_variant_from_trace(log[i])))]):
            next
        else:
            variants_deviating['deviating'][','.join(list(get_variant_from_trace(log[i])))] = min(1, sum(
                collect_traces[d][i] for d in dev))
            variants_deviating['first_idx'][','.join(list(get_variant_from_trace(log[i])))] = i
            variants_deviating['count'][','.join(list(get_variant_from_trace(log[i])))] = var_counts[
                get_variant_from_trace(log[i])]
            if REL_INPUT_PATH == "/BINet/":
                variants_deviating['label'][','.join(list(get_variant_from_trace(log[i])))] = log[i].attributes['label']
    vars_to_analyze = []
    for var in variant_list:
        if variants_deviating['deviating'][var] == 1:
            vars_to_analyze.append(
                var)  # this list stores all variants that deviate, i.e., all variants that should be analyzed

    parameters = {}

    # we now extract AND- and XOR-block from the model
    debug = exec_utils.get_param_value(Parameters.DEBUG, parameters, False)
    fold = exec_utils.get_param_value(Parameters.FOLD, parameters, True)
    grouped_net = pm4py.objects.conversion.wf_net.variants.to_process_tree.group_blocks_in_net(net,
                                                                                               parameters=parameters)  # this function finds all block-structures in form of process-tree labels for transitions (one transition corresponds to one block-structure)
    pn_viz.view(pn_viz.apply(grouped_net))

    parallels = {}
    exclusives = {}
    acts = []
    for t in range(len(list(net.transitions))):
        acts.append(list(net.transitions)[t].label)
    for trans in range(len(list(
            grouped_net.transitions))):  # here, we iterate over all labels and collect tree parts with XOR- and AND-operator
        if not list(grouped_net.transitions)[trans].label is None and not str(
                list(grouped_net.transitions)[trans].label) in acts:
            pt_str = list(grouped_net.transitions)[trans].label
            pt = pt_util.parse(pt_str)
            tree_part = pt_util.fold(pt) if fold else pt
            tree_sort(tree_part)
            parallels, exclusives = relations(tree_part, parallels, exclusives)

    solution_patterns = pd.DataFrame(
        index=variants_deviating.index)  # this stores the solution patterns, i.e., which will be the decisions for each variant
    done = 0
    manager_patterns = pd.DataFrame(index=variants_deviating.index)  # this stores the post-processed results

    for var in vars_to_analyze:
        dev_in_var = []
        i = variants_deviating['first_idx'][var]
        for j in range(0, len(aligned_traces[i]['alignment'])):
            if aligned_traces[i]['alignment'][j][1] == None or aligned_traces[i]['alignment'][j][0] == \
                    aligned_traces[i]['alignment'][j][1]:
                next  # we do not care for simultaneous or silent moves
            else:
                if aligned_traces[i]['alignment'][j][1] == '>>':
                    dev_in_var.append(str('log_' + str(j) + '_' + aligned_traces[i]['alignment'][j][0]))
                elif aligned_traces[i]['alignment'][j][0] == '>>':
                    dev_in_var.append(str('model_' + str(j) + '_' + aligned_traces[i]['alignment'][j][1]))

        devs_info = pd.DataFrame(data=None, columns=['activity', 'type', 'alignment_position'],
                                 index=dev_in_var)  # find and store all deviations in the variant
        for j in range(0, len(aligned_traces[i]['alignment'])):
            if aligned_traces[i]['alignment'][j][1] == None or aligned_traces[i]['alignment'][j][0] == \
                    aligned_traces[i]['alignment'][j][1]:
                next  # we do not care for simultaneous or silent moves
            else:
                if aligned_traces[i]['alignment'][j][1] == '>>':
                    devs_info['activity'][str('log_' + str(j) + '_' + aligned_traces[i]['alignment'][j][0])] = \
                        aligned_traces[i]['alignment'][j][0]
                    devs_info['type'][str('log_' + str(j) + '_' + aligned_traces[i]['alignment'][j][0])] = 'log'
                    devs_info['alignment_position'][str('log_' + str(j) + '_' + aligned_traces[i]['alignment'][j][0])] = j
                elif aligned_traces[i]['alignment'][j][0] == '>>':
                    devs_info['activity'][str('model_' + str(j) + '_' + aligned_traces[i]['alignment'][j][1])] = \
                        aligned_traces[i]['alignment'][j][1]
                    devs_info['type'][str('model_' + str(j) + '_' + aligned_traces[i]['alignment'][j][1])] = 'model'
                    devs_info['alignment_position'][str('model_' + str(j) + '_' + aligned_traces[i]['alignment'][j][1])] = j

        swap_patterns = {}
        replace_patterns = {}
        repeated_patterns = {}
        repl = 1
        repea = 1
        swa = 1
        swapped_patterns_single = []
        replace_patterns_single = []
        repeated_patterns_single = []
        # now we extract all instantiations of deviation patterns (this part is referred to as Algorithm 1 in the paper)
        for i, d in enumerate(dev_in_var):
            #swapped
            swapped_block1 = []
            dev_block1 = []
            count = devs_info['activity'].value_counts()[devs_info['activity'][d]]
            typ_1 = devs_info['type'][d]
            if count > 1:
                next_dev_with_activity = False
                swapped_block1.append(devs_info['activity'][d])
                dev_block1.append(d)
                j = i
                while j + 1 < len(dev_in_var):
                    j += 1

                    if typ_1 == devs_info['type'][dev_in_var[j]] and (
                            devs_info['alignment_position'][dev_in_var[j - 1]] == devs_info['alignment_position'][
                        dev_in_var[j]] - 1 or all(
                        aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] == None for run in
                        range(devs_info['alignment_position'][dev_in_var[j - 1]], devs_info['alignment_position'][
                            dev_in_var[j]]))):
                        swapped_block1.append(devs_info['activity'][dev_in_var[j]])
                        dev_block1.append(dev_in_var[j])
                    else:
                        j = 100000000
                j = i

                while next_dev_with_activity == False and j + 1 < len(dev_in_var):
                    j += 1
                    if devs_info['activity'][dev_in_var[i]] == devs_info['activity'][dev_in_var[j]] and not \
                    devs_info['type'][
                        dev_in_var[
                            i]] == \
                    devs_info['type'][
                        dev_in_var[
                            j]]:  # yes if there is log and model move on same activity
                        too_many_block1 = []
                        dev_block2 = [dev_in_var[j]]
                        for check_runner in range(j + 1, j + len(dev_block1)):
                            if check_runner >= len(dev_in_var):
                                next
                            elif devs_info['alignment_position'][dev_in_var[check_runner]] >= len(
                                    aligned_traces[variants_deviating['first_idx'][var]]['alignment']):
                                next
                            elif not devs_info['alignment_position'][dev_in_var[check_runner]] < len(
                                    aligned_traces[variants_deviating['first_idx'][var]]['alignment']):
                                del dev_block1[-(check_runner - j):]
                                del swapped_block1[-(check_runner - j):]
                                break
                            elif not ((aligned_traces[variants_deviating['first_idx'][var]]['alignment'][
                                           devs_info['alignment_position'][dev_in_var[check_runner]]][1] == '>>' and
                                       aligned_traces[variants_deviating['first_idx'][var]]['alignment'][
                                           devs_info['alignment_position'][dev_in_var[check_runner]]][0] == swapped_block1[
                                           check_runner - j]) or (
                                              aligned_traces[variants_deviating['first_idx'][var]]['alignment'][
                                                  devs_info['alignment_position'][dev_in_var[check_runner]]][0] == '>>' and
                                              aligned_traces[variants_deviating['first_idx'][var]]['alignment'][
                                                  devs_info['alignment_position'][dev_in_var[check_runner]]][1] ==
                                              swapped_block1[check_runner - j])):
                                too_many_block1.append(check_runner - j)
                            else:
                                dev_block2.append(dev_in_var[check_runner])
                        for index in range(len(swapped_block1) - 1, -1, -1):
                            if not swapped_block1[index] in ''.join(dev_block2):
                                del swapped_block1[index]
                                del dev_block1[index]
                        next_dev_with_activity = not next_dev_with_activity
                        swapped_activities = []
                        for move in range(devs_info['alignment_position'][dev_in_var[i]] + 1,
                                          devs_info['alignment_position'][dev_in_var[j]]):
                            if not str(aligned_traces[variants_deviating['first_idx'][var]]['alignment'][move][
                                           0]) == '>>' and not str(
                                    aligned_traces[variants_deviating['first_idx'][var]]['alignment'][move][
                                        0]) in swapped_block1:
                                swapped_activities.append(
                                    str(aligned_traces[variants_deviating['first_idx'][var]]['alignment'][move][0]))
                        idexes = []
                        # this annotates deviation patterns
                        for missing in range(len(swapped_activities)):
                            for par in range(len(parallels.keys())):
                                for p in range(len(parallels[list(parallels.keys())[par]])):
                                    if str(parallels[list(parallels.keys())[par]][p]) == str(swapped_activities[missing]):
                                        print(str(parallels[list(parallels.keys())[par]][p]) == str(
                                            swapped_activities[missing]), swapped_activities[missing],
                                              parallels[list(parallels.keys())[par]][p])
                                        idexes.append(missing)
                                print(len(idexes), len(parallels[list(parallels.keys())[par]]))
                                if len(idexes) == len(parallels[list(parallels.keys())[par]]):
                                    print(1)
                                    for i in sorted(idexes, reverse=True):
                                        if i == idexes[0]:
                                            swapped_activities[idexes[0]] = str(
                                                '+(' + str(parallels[list(parallels.keys())[par]]) + ')')
                                        else:
                                            del swapped_activities[i]
                        idexes = []

                        if len(swapped_block1) > 0 and len(swapped_activities) > 0:
                            if devs_info['type'][dev_in_var[i]] == 'log':
                                if not str('swap_' + ', '.join(sorted(swapped_activities)) + ' and ' + ', '.join(
                                        swapped_block1)) in swapped_patterns_single:
                                    swapped_patterns_single.append(
                                        str('swap_' + ', '.join(sorted(swapped_activities)) + ' and ' + ', '.join(
                                            swapped_block1)))
                                if not str(str(swa) + 'swap_' + ', '.join(sorted(swapped_activities)) + ' and ' + ', '.join(
                                        swapped_block1)) in swap_patterns.keys():
                                    swap_patterns[
                                        str(str(swa) + 'swap_' + ', '.join(
                                            sorted(swapped_activities)) + ' and ' + ', '.join(swapped_block1))] = str(
                                        '-'.join(dev_block1) + '-' + '-'.join(dev_block2))
                                    swa += 1
                            elif devs_info['type'][dev_in_var[i]] == 'model':
                                if not str('swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                        sorted(swapped_activities))) in swapped_patterns_single:
                                    swapped_patterns_single.append(
                                        str('swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                            sorted(swapped_activities))))
                                if not str(str(swa) + 'swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                        sorted(swapped_activities))) in swap_patterns.keys():
                                    swap_patterns[str(str(swa) + 'swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                        sorted(swapped_activities)))] = str(
                                        '-'.join(dev_block1) + '-' + '-'.join(dev_block2))
                                    swa += 1

            act_1 = devs_info['activity'][d]
            for excl in range(len(exclusives.keys())):
                for ex in range(len(exclusives[list(exclusives.keys())[excl]])):
                    if str(exclusives[list(exclusives.keys())[excl]][ex]) == str(act_1):
                        swapped_block1.append(devs_info['activity'][d])
                        dev_block1.append(d)
                        dev_block2 = []
                        j = i
                        while j + 1 < len(dev_in_var):
                            j += 1
                            act_2 = devs_info['activity'][dev_in_var[j]]
                            typ_2 = devs_info['type'][dev_in_var[j]]
                            for excl9 in range(len(exclusives.keys())):
                                for ex9 in range(len(exclusives[list(exclusives.keys())[excl9]])):
                                    if str(exclusives[list(exclusives.keys())[excl9]][ex9]) == str(
                                            act_2) and typ_2 == typ_1 and not dev_in_var[j] in dev_block1:
                                        dev_block1.append(dev_in_var[j])
                                        swapped_block1.append(devs_info['activity'][dev_in_var[j]])
                            for ex2 in range(len(exclusives[list(exclusives.keys())[excl]])):
                                if str(exclusives[list(exclusives.keys())[excl]][ex2]) == str(act_2) and typ_2 != typ_1:
                                    dev_block2.append(dev_in_var[j])
                                    swapped_activities = []
                                    for move in range(devs_info['alignment_position'][dev_in_var[i]] + 1,
                                                      devs_info['alignment_position'][dev_in_var[j]]):
                                        if not str(aligned_traces[variants_deviating['first_idx'][var]]['alignment'][move][
                                                       0]) == '>>' and not str(
                                                aligned_traces[variants_deviating['first_idx'][var]]['alignment'][move][
                                                    0]) in swapped_block1:
                                            swapped_activities.append(
                                                str(aligned_traces[variants_deviating['first_idx'][var]]['alignment'][move][
                                                        0]))

                                    if len(swapped_block1) > 0 and len(swapped_activities) > 0:
                                        if devs_info['type'][dev_in_var[i]] == 'log':
                                            if not str(
                                                    'swap_' + ', '.join(sorted(swapped_activities)) + ' and ' + ', '.join(
                                                            swapped_block1)) in swapped_patterns_single:
                                                swapped_patterns_single.append(str('swap_' + ', '.join(
                                                    sorted(swapped_activities)) + ' and ' + ', '.join(swapped_block1)))
                                            if not str(str(swa) + 'swap_' + ', '.join(
                                                    sorted(swapped_activities)) + ' and ' + ', '.join(
                                                    swapped_block1)) in swap_patterns.keys():
                                                swap_patterns[
                                                    str(str(swa) + 'swap_' + ', '.join(
                                                        sorted(swapped_activities)) + ' and ' + ', '.join(
                                                        swapped_block1))] = str(
                                                    '-'.join(dev_block1) + '-' + '-'.join(dev_block2))
                                                swa += 1
                                        elif devs_info['type'][dev_in_var[i]] == 'model':
                                            if not str('swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                                    sorted(swapped_activities))) in swapped_patterns_single:
                                                swapped_patterns_single.append(
                                                    str('swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                                        sorted(swapped_activities))))
                                            if not str(str(swa) + 'swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                                    sorted(swapped_activities))) in swap_patterns.keys():
                                                swap_patterns[str(str(swa) + 'swap_' + ', '.join(
                                                    swapped_block1) + ' and ' + ', '.join(
                                                    sorted(swapped_activities)))] = str(
                                                    '-'.join(dev_block1) + '-' + '-'.join(dev_block2))
                                                swa += 1
            #replaced
            if len(devs_info) - i > 2 and (
                    devs_info['alignment_position'][dev_in_var[i]] == devs_info['alignment_position'][
                dev_in_var[i + 1]] - 1 or all(
                aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] == None for run in
                range(devs_info['alignment_position'][dev_in_var[i]], devs_info['alignment_position'][
                    dev_in_var[i + 1]]))):
                consequetive_devs = 2
                multi_log = []
                multi_model = []
                multi_log_devs = []
                multi_model_devs = []
                for runner in range(i + 2, len(devs_info)):
                    if (devs_info['alignment_position'][dev_in_var[runner - 1]] == devs_info['alignment_position'][
                        dev_in_var[runner]] - 1 or all(
                        aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] == None for run in
                        range(devs_info['alignment_position'][dev_in_var[runner - 1]], devs_info['alignment_position'][
                            dev_in_var[runner]]))):
                        consequetive_devs += 1
                    else:
                        break
                for z in range(i, i + consequetive_devs):
                    if devs_info['type'][dev_in_var[z]] == 'model':
                        multi_model.append(devs_info['activity'][dev_in_var[z]])
                        multi_model_devs.append(str(dev_in_var[z]))
                    elif devs_info['type'][dev_in_var[z]] == 'log':
                        multi_log.append(devs_info['activity'][dev_in_var[z]])
                        multi_log_devs.append(str(dev_in_var[z]))

                idexes = []
                for missing in range(len(multi_model)):
                    for excl in range(len(exclusives.keys())):
                        for ex in range(len(exclusives[list(exclusives.keys())[excl]])):
                            if str(exclusives[list(exclusives.keys())[excl]][ex]) == str(multi_model[missing]):
                                print(str(exclusives[list(exclusives.keys())[excl]][ex]) == str(multi_model[missing]),
                                      multi_model[missing], exclusives[list(exclusives.keys())[excl]][ex])
                                multi_model[missing] = str('x(' + str(exclusives[list(exclusives.keys())[excl]]) + ')')
                    for par in range(len(parallels.keys())):
                        for p in range(len(parallels[list(parallels.keys())[par]])):
                            if str(parallels[list(parallels.keys())[par]][p]) == str(multi_model[missing]):
                                print(str(parallels[list(parallels.keys())[par]][p]) == str(multi_model[missing]),
                                      multi_model[missing], parallels[list(parallels.keys())[par]][p])
                                idexes.append(missing)
                        print(len(idexes), len(parallels[list(parallels.keys())[par]]))
                        if len(idexes) == len(parallels[list(parallels.keys())[par]]):
                            print(1)
                            for i in sorted(idexes, reverse=True):
                                if i == idexes[0]:
                                    multi_model[idexes[0]] = str('+(' + str(parallels[list(parallels.keys())[par]]) + ')')
                                else:
                                    del multi_model[i]

                if not (len(multi_log) == 0 or len(multi_model) == 0):
                    if not str(
                            'replace_' + ', '.join(multi_model) + ' by ' + ', '.join(multi_log)) in replace_patterns_single:
                        replace_patterns_single.append(
                            str('replace_' + ', '.join(multi_model) + ' by ' + ', '.join(multi_log)))
                    replace_patterns[
                        str(str(repl) + 'replace_' + ', '.join(multi_model) + ' by ' + ', '.join(multi_log))] = str(
                        '-'.join(multi_model_devs) + '_' + '-'.join(multi_log_devs))
                    repl = +1
            elif len(devs_info) - i > 1 and (
                    devs_info['alignment_position'][dev_in_var[i]] == devs_info['alignment_position'][
                dev_in_var[i + 1]] - 1 or all(
                aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] == None for run in
                range(devs_info['alignment_position'][dev_in_var[i]], devs_info['alignment_position'][
                    dev_in_var[i + 1]]))) and not devs_info['type'][dev_in_var[i]] == devs_info['type'][dev_in_var[i + 1]]:
                j = i + 1
                possible_multi_type = devs_info['type'][dev_in_var[j]]
                possible_multi_acts = [devs_info['activity'][dev_in_var[j]]]
                multi_devs = [str(dev_in_var[j])]
                while j < len(devs_info) - 1:
                    if devs_info['type'][dev_in_var[j]] == devs_info['type'][dev_in_var[j + 1]] and (
                            devs_info['alignment_position'][dev_in_var[j]] == devs_info['alignment_position'][
                        dev_in_var[j + 1]] - 1 or all(
                        aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] == None for run in
                        range(devs_info['alignment_position'][dev_in_var[j]], devs_info['alignment_position'][
                            dev_in_var[j + 1]]))):
                        possible_multi_acts.append(devs_info['activity'][dev_in_var[j + 1]])
                        multi_devs.append(str(dev_in_var[j + 1]))
                        j += 1
                    else:
                        j = 100000
                if len(multi_devs) <= 1:
                    if devs_info['type'][dev_in_var[i]] == 'log':
                        log_act = devs_info['activity'][dev_in_var[i]]
                        model_act = devs_info['activity'][dev_in_var[i + 1]]
                    elif devs_info['type'][dev_in_var[i]] == 'model':
                        log_act = devs_info['activity'][dev_in_var[i + 1]]
                        model_act = devs_info['activity'][dev_in_var[i]]
                    if not str('replace_' + model_act + ' by ' + log_act) in replace_patterns_single:
                        replace_patterns_single.append(str('replace_' + model_act + ' by ' + log_act))
                    replace_patterns[str(str(repl) + 'replace_' + model_act + ' by ' + log_act)] = str(
                        dev_in_var[i] + '-' + dev_in_var[i + 1])
                    repl = +1
                else:
                    if devs_info['type'][dev_in_var[i]] == 'log':
                        log_act = devs_info['activity'][dev_in_var[i]]

                        idexes = []
                        for missing in range(len(possible_multi_acts)):
                            for excl in range(len(exclusives.keys())):
                                for ex in range(len(exclusives[list(exclusives.keys())[excl]])):
                                    if str(exclusives[list(exclusives.keys())[excl]][ex]) == str(
                                            possible_multi_acts[missing]):
                                        print(str(exclusives[list(exclusives.keys())[excl]][ex]) == str(
                                            possible_multi_acts[missing]), possible_multi_acts[missing],
                                              exclusives[list(exclusives.keys())[excl]][ex])
                                        possible_multi_acts[missing] = str(
                                            'x(' + str(exclusives[list(exclusives.keys())[excl]]) + ')')
                            for par in range(len(parallels.keys())):
                                for p in range(len(parallels[list(parallels.keys())[par]])):
                                    if str(parallels[list(parallels.keys())[par]][p]) == str(possible_multi_acts[missing]):
                                        print(str(parallels[list(parallels.keys())[par]][p]) == str(
                                            possible_multi_acts[missing]), possible_multi_acts[missing],
                                              parallels[list(parallels.keys())[par]][p])
                                        idexes.append(missing)
                                print(len(idexes), len(parallels[list(parallels.keys())[par]]))
                                if len(idexes) == len(parallels[list(parallels.keys())[par]]):
                                    print(1)
                                    for i in sorted(idexes, reverse=True):
                                        if i == idexes[0]:
                                            possible_multi_acts[idexes[0]] = str(
                                                '+(' + str(parallels[list(parallels.keys())[par]]) + ')')
                                        else:
                                            del possible_multi_acts[i]
                        model_act = ', '.join(possible_multi_acts)
                    elif devs_info['type'][dev_in_var[i]] == 'model':
                        log_act = ', '.join(possible_multi_acts)
                        model_act = devs_info['activity'][dev_in_var[i]]
                    if not str('replace_' + model_act + ' by ' + log_act) in replace_patterns_single:
                        replace_patterns_single.append(str('replace_' + model_act + ' by ' + log_act))
                    replace_patterns[str(str(repl) + 'replace_' + model_act + ' by ' + log_act)] = str(
                        dev_in_var[i] + '-' + '-'.join(multi_devs))
                    repl = +1

            #repeated
            if devs_info['type'][dev_in_var[i]] == 'log' and devs_info['activity'][dev_in_var[i]] in list(
                    aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][0] for run in
                    range(devs_info['alignment_position'][dev_in_var[i]])) and devs_info['activity'][dev_in_var[i]] in list(
                aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] for run in
                range(devs_info['alignment_position'][dev_in_var[i]])):
                if len(devs_info) - i >= 1:
                    potential_multi = [devs_info['activity'][dev_in_var[i]]]
                    multi_devs = [str(dev_in_var[i])]
                    for multi in range(i + 1, len(devs_info)):
                        if not devs_info['type'][dev_in_var[multi]] == 'log' or not \
                                aligned_traces[variants_deviating['first_idx'][var]]['alignment'][
                                    devs_info['alignment_position'][dev_in_var[multi]]][1] == '>>' or not \
                        devs_info['activity'][
                            dev_in_var[
                                multi]] in list(
                            aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][0] for run in
                            range(devs_info['alignment_position'][dev_in_var[multi]])) or not (
                                devs_info['alignment_position'][dev_in_var[multi - 1]] == devs_info['alignment_position'][
                            dev_in_var[multi]] - 1 or all(
                            aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] == None for run in
                            range(devs_info['alignment_position'][dev_in_var[multi - 1]], devs_info['alignment_position'][
                                dev_in_var[multi]]))):
                            break
                        else:
                            potential_multi.append(devs_info['activity'][dev_in_var[multi]])
                            multi_devs.append(str(dev_in_var[multi]))
                            multi_length = multi - i
                    if not str('repeated_' + ' and '.join(potential_multi)) in repeated_patterns_single:
                        repeated_patterns_single.append(str('repeated_' + ' and '.join(potential_multi)))
                    repeated_patterns[str(str(repea) + 'repeated_' + ' and '.join(potential_multi))] = '-'.join(multi_devs)

                    repea += 1

        for key in swap_patterns.keys():
            if not key in solution_patterns.columns:
                solution_patterns[key] = 0
        for key in replace_patterns.keys():
            if not key in solution_patterns.columns:
                solution_patterns[key] = 0
        for key in repeated_patterns.keys():
            if not key in solution_patterns.columns:
                solution_patterns[key] = 0
        for key in dev_in_var:
            if not key in solution_patterns.columns:
                solution_patterns[key] = 0

        # from here on, we build the linear program
        model = Model(name="solution_variants")
        indiv_devs = model.binary_var_list(dev_in_var)
        swap_pattern_devs = model.binary_var_list(list(swap_patterns.keys()))
        replace_pattern_devs = model.binary_var_list(list(replace_patterns.keys()))
        repeated_pattern_devs = model.binary_var_list(list(repeated_patterns.keys()))
        model.add_constraints(model.sum(indiv_devs[i]) + model.sum(
            ((indiv_devs[i].name in swap_patterns[swap_pattern_devs[j].name]) * 1 * swap_pattern_devs[j]) for j in
            range(len(swap_patterns.keys()))) + model.sum(
            ((indiv_devs[i].name in replace_patterns[replace_pattern_devs[j].name]) * 1 * replace_pattern_devs[j]) for j in
            range(len(replace_patterns.keys()))) + model.sum(
            ((indiv_devs[i].name in repeated_patterns[repeated_pattern_devs[j].name]) * 1 * repeated_pattern_devs[j]) for j
            in
            range(len(repeated_patterns.keys()))) == 1 for i in range(len(dev_in_var)))
        objective = model.sum(indiv_devs[i] for i in range(len(dev_in_var))) * bigM + model.sum(
            swap_pattern_devs[j] for j in range(len(swap_pattern_devs))) * cost_swap + model.sum(
            replace_pattern_devs[j] for j in range(len(replace_pattern_devs))) * cost_replace + model.sum(
            repeated_pattern_devs[j] for j in range(len(repeated_pattern_devs))) * cost_repeat
        model.minimize(objective)
        model.solve()
        model.solve_details
        for j in indiv_devs:
            solution_patterns[j.name][var] = j.solution_value
        for j in swap_pattern_devs:
            solution_patterns[j.name][var] = j.solution_value
        for j in replace_pattern_devs:
            solution_patterns[j.name][var] = j.solution_value
        for j in repeated_pattern_devs:
            solution_patterns[j.name][var] = j.solution_value
        done += 1

        # post-processing starts here
        solution_patterns['deviating'] = variants_deviating['deviating']
        manager_patterns['deviating'] = solution_patterns['deviating']
        manager_patterns['count'] = variants_deviating['count']
        manager_patterns['label'] = variants_deviating['label']

        for single in swapped_patterns_single:
            for key in list(swap_patterns.keys()):
                if single in key and len(single) == len(key) - 1:
                    if not single in manager_patterns.keys():
                        manager_patterns[single] = 0
                    manager_patterns[single][var] += solution_patterns[key][var]
        for single in replace_patterns_single:
            for key in list(replace_patterns.keys()):
                if single in key and len(single) == len(key) - 1:
                    if not single in manager_patterns.keys():
                        manager_patterns[single] = 0
                    manager_patterns[single][var] += solution_patterns[key][var]
        for single in repeated_patterns_single:
            for key in list(repeated_patterns.keys()):
                if single in key and len(single) == len(key) - 1:
                    if not single in manager_patterns.keys():
                        manager_patterns[single] = 0
                    manager_patterns[single][var] += solution_patterns[key][var]
        used_moves = 0

        for i, dev_idx in enumerate(list(devs_info.index)):

            if len(devs_info) - i > 1:
                if solution_patterns[dev_idx][var] == 1 and devs_info['alignment_position'][dev_idx] > used_moves and \
                        devs_info['alignment_position'][list(devs_info.index)[i]] == devs_info['alignment_position'][
                    list(devs_info.index)[i + 1]] - 1:
                    list_activities = [str(devs_info['activity'][dev_idx])]
                    j = i + 1
                    if j < len(devs_info):
                        while j <= len(list(devs_info.index)) - 1:
                            if solution_patterns[list(devs_info.index)[j]][var] == 1 and devs_info['type'][dev_idx] == \
                                    devs_info['type'][list(devs_info.index)[j]] and devs_info['alignment_position'][
                                list(devs_info.index)[j]] == devs_info['alignment_position'][
                                list(devs_info.index)[j - 1]] + 1:
                                list_activities.append(str(devs_info['activity'][list(devs_info.index)[j]]))
                                used_moves = devs_info['alignment_position'][list(devs_info.index)[j]]
                            j += 1
                    if devs_info['type'][dev_idx] == 'log':
                        type = 'inserted_'
                    else:
                        type = 'missing_'
                    if type == 'missing_':
                        idexes = []
                        for missing in range(len(list_activities)):
                            for excl in range(len(exclusives.keys())):
                                for ex in range(len(exclusives[list(exclusives.keys())[excl]])):
                                    if str(exclusives[list(exclusives.keys())[excl]][ex]) == str(list_activities[missing]):
                                        print(str(exclusives[list(exclusives.keys())[excl]][ex]) == str(
                                            list_activities[missing]), list_activities[missing],
                                              exclusives[list(exclusives.keys())[excl]][ex])
                                        list_activities[missing] = str(
                                            'x(' + str(exclusives[list(exclusives.keys())[excl]]) + ')')
                            for par in range(len(parallels.keys())):
                                for p in range(len(parallels[list(parallels.keys())[par]])):
                                    if str(parallels[list(parallels.keys())[par]][p]) == str(list_activities[missing]):
                                        print(
                                            str(parallels[list(parallels.keys())[par]][p]) == str(list_activities[missing]),
                                            list_activities[missing], parallels[list(parallels.keys())[par]][p])
                                        idexes.append(missing)
                                print(len(idexes), len(parallels[list(parallels.keys())[par]]))
                                if len(idexes) == len(parallels[list(parallels.keys())[par]]):
                                    print(1)
                                    for i in sorted(idexes, reverse=True):
                                        if i == idexes[0]:
                                            list_activities[idexes[0]] = str(
                                                '+(' + str(parallels[list(parallels.keys())[par]]) + ')')
                                        else:
                                            del list_activities[i]
                    print(1, i, list_activities)
                    cluster = str(type + ', '.join(list_activities))
                    if not cluster in manager_patterns.columns:
                        manager_patterns[cluster] = 0
                    manager_patterns[cluster][var] += 1
                elif solution_patterns[dev_idx][var] == 1 and devs_info['alignment_position'][dev_idx] > used_moves:
                    list_activities = [str(devs_info['activity'][dev_idx])]
                    if devs_info['type'][dev_idx] == 'log':
                        type = 'inserted_'
                    else:
                        type = 'missing_'
                    if type == 'missing_':
                        idexes = []
                        for missing in range(len(list_activities)):
                            for excl in range(len(exclusives.keys())):
                                for ex in range(len(exclusives[list(exclusives.keys())[excl]])):
                                    if str(exclusives[list(exclusives.keys())[excl]][ex]) == str(list_activities[missing]):
                                        print(str(exclusives[list(exclusives.keys())[excl]][ex]) == str(
                                            list_activities[missing]), list_activities[missing],
                                              exclusives[list(exclusives.keys())[excl]][ex])
                                        list_activities[missing] = str(
                                            'x(' + str(exclusives[list(exclusives.keys())[excl]]) + ')')
                            for par in range(len(parallels.keys())):
                                for p in range(len(parallels[list(parallels.keys())[par]])):
                                    if str(parallels[list(parallels.keys())[par]][p]) == str(list_activities[missing]):
                                        print(
                                            str(parallels[list(parallels.keys())[par]][p]) == str(list_activities[missing]),
                                            list_activities[missing], parallels[list(parallels.keys())[par]][p])
                                        idexes.append(missing)
                                print(len(idexes), len(parallels[list(parallels.keys())[par]]))
                                if len(idexes) == len(parallels[list(parallels.keys())[par]]):
                                    print(1)
                                    for i in sorted(idexes, reverse=True):
                                        if i == idexes[0]:
                                            list_activities[idexes[0]] = str(
                                                '+(' + str(parallels[list(parallels.keys())[par]]) + ')')
                                        else:
                                            del list_activities[i]
                    print(2, list_activities)
                    cluster = str(type + ', '.join(list_activities))
                    if not cluster in manager_patterns.columns:
                        manager_patterns[cluster] = 0
                    manager_patterns[cluster][var] += 1
                    used_moves = devs_info['alignment_position'][list(devs_info.index)[i]]
            elif solution_patterns[dev_idx][var] == 1 and (
                    devs_info['alignment_position'][dev_idx] > used_moves or devs_info['alignment_position'][
                dev_idx] == 0 or len(devs_info) == 1):
                list_activities = [str(devs_info['activity'][dev_idx])]
                if devs_info['type'][dev_idx] == 'log':
                    type = 'inserted_'
                else:
                    type = 'missing_'
                if type == 'missing_':
                    idexes = []
                    for missing in range(len(list_activities)):
                        for excl in range(len(exclusives.keys())):
                            for ex in range(len(exclusives[list(exclusives.keys())[excl]])):
                                if str(exclusives[list(exclusives.keys())[excl]][ex]) == str(list_activities[missing]):
                                    print(
                                        str(exclusives[list(exclusives.keys())[excl]][ex]) == str(list_activities[missing]),
                                        list_activities[missing], exclusives[list(exclusives.keys())[excl]][ex])
                                    list_activities[missing] = str(
                                        'x(' + str(exclusives[list(exclusives.keys())[excl]]) + ')')
                        for par in range(len(parallels.keys())):
                            for p in range(len(parallels[list(parallels.keys())[par]])):
                                if str(parallels[list(parallels.keys())[par]][p]) == str(list_activities[missing]):
                                    print(str(parallels[list(parallels.keys())[par]][p]) == str(list_activities[missing]),
                                          list_activities[missing], parallels[list(parallels.keys())[par]][p])
                                    idexes.append(missing)
                            print(len(idexes), len(parallels[list(parallels.keys())[par]]))
                            if len(idexes) == len(parallels[list(parallels.keys())[par]]):
                                print(1)
                                for i in sorted(idexes, reverse=True):
                                    if i == idexes[0]:
                                        list_activities[idexes[0]] = str(
                                            '+(' + str(parallels[list(parallels.keys())[par]]) + ')')
                                    else:
                                        del list_activities[i]
                cluster = str(type + ', '.join(list_activities))
                if not cluster in manager_patterns.columns:
                    manager_patterns[cluster] = 0
                manager_patterns[cluster][var] += 1
                used_moves = devs_info['alignment_position'][list(devs_info.index)[i]]
    solution_patterns = solution_patterns.sort_index(axis=1).drop(['deviating'], axis=1)

    path = (os.getcwd() + '/Evaluation_opt')
    timer = pd.DataFrame(data=0, columns=['time'], index=[0])
    timer['time'][0] = time.clock() - time_start

    xt, z = os.path.split(file)
    writer = pd.ExcelWriter(
        path + '/2new_' + z + '.xlsx',
        engine="xlsxwriter")


    timer.to_excel(writer, sheet_name=('Time'))
    export_solutions = pd.DataFrame(index=solution_patterns.index, columns=['deviating', 'count', 'label'])
    export_solutions['deviating'] = manager_patterns['deviating']
    export_solutions['count'] = manager_patterns['count']
    export_solutions['label'] = manager_patterns['label']
    export_solutions
    manager_patterns = manager_patterns.sort_index(axis=1, ascending=False)
    mgt_pts = manager_patterns.drop(columns=['deviating', 'count', 'label'])
    for i, index in enumerate(list(export_solutions.index)):
        counter = 0
        for col in mgt_pts.columns:
            if not pd.isna(manager_patterns['label'][index]) and counter > 0:
                continue
            if mgt_pts[col][index] > 0:
                if not str('pld_' + str(counter)) in export_solutions.columns:
                    export_solutions[str('pld_' + str(counter))] = 0
                if not str('count_pld_' + str(counter)) in export_solutions.columns:
                    export_solutions[str('count_pld_' + str(counter))] = 0
                export_solutions[str('pld_' + str(counter))][index] = col
                export_solutions[str('count_pld_' + str(counter))][index] += mgt_pts[col][index]
                counter += 1
    export_solutions.to_excel(writer, sheet_name=('export'))
    writer.close()
    return manager_patterns, variants_deviating, solution_patterns

In [ ]:
manager_patterns, variants_deviating, solution_patterns=discover_patterns(log, net, initial_marking, final_marking)

In [ ]:
def discover_patterns_wo_cf(log, net, initial_marking, final_marking, bigM=1000, cost_swap=1, cost_replace=1.1, cost_repeat=1.2):
    xt, z = os.path.split(file)
    time_start = time.clock()
    aligned_traces = generate_alignments_pkl(log, net, initial_marking, final_marking)
    #### get information whether deviation happened after prefix length in DF
    max_moves = 0
    dev = []  # stores all deviations that happened
    for i, trace in enumerate(log):
        no_moves = len(aligned_traces[i]['alignment'])
        if max_moves < no_moves:
            max_moves = no_moves
        for j in range(0, len(aligned_traces[i]['alignment'])):
            if aligned_traces[i]['alignment'][j][1] == None or aligned_traces[i]['alignment'][j][0] == \
                    aligned_traces[i]['alignment'][j][1]:
                next  # we do not care for simultaneous or silent moves
            else:
                if not str(aligned_traces[i]['alignment'][j]) in dev:
                    dev.append(str(aligned_traces[i]['alignment'][j]))

    collect_traces = pd.DataFrame(data=0, columns=dev, index=range(
        len(log)))  # Data Frame that stores the information whether a deviation happened for each trace on trace level
    collect_traces['variant_idx'] = None

    # select only one trace per variant
    variants = pm4py.get_variants(log)
    variant_list = list(variants.keys())
    variant_list = list(','.join(variant_list[e]) for e in range(len(variant_list)))

    for i, trace in enumerate(log):
        collect_traces['variant_idx'][i] = variant_list.index(','.join(list(get_variant_from_trace(log[i]))))
        for j in range(0, len(aligned_traces[i]['alignment'])):
            if aligned_traces[i]['alignment'][j][1] == None or aligned_traces[i]['alignment'][j][0] == \
                    aligned_traces[i]['alignment'][j][1]:
                next
            else:
                collect_traces[str(aligned_traces[i]['alignment'][j])][i] = 1

    var_counts = dict(get_variants_sorted_by_count(variants))
    variants_deviating = pd.DataFrame(data=None, columns=['deviating', 'first_idx', 'count', 'label'], index=variant_list)
    for i in range(len(log)):
        if not pd.isna(variants_deviating['first_idx'][','.join(list(get_variant_from_trace(log[i])))]):
            next
        else:
            variants_deviating['deviating'][','.join(list(get_variant_from_trace(log[i])))] = min(1, sum(
                collect_traces[d][i] for d in dev))
            variants_deviating['first_idx'][','.join(list(get_variant_from_trace(log[i])))] = i
            variants_deviating['count'][','.join(list(get_variant_from_trace(log[i])))] = var_counts[
                get_variant_from_trace(log[i])]
            if REL_INPUT_PATH == "/BINet/":
                variants_deviating['label'][','.join(list(get_variant_from_trace(log[i])))] = log[i].attributes['label']
    vars_to_analyze = []
    for var in variant_list:
        if variants_deviating['deviating'][var] == 1:
            vars_to_analyze.append(
                var)  # this list stores all variants that deviate, i.e., all variants that should be analyzed

    parameters = {}

    # we now extract AND- and XOR-block from the model
    debug = exec_utils.get_param_value(Parameters.DEBUG, parameters, False)
    fold = exec_utils.get_param_value(Parameters.FOLD, parameters, True)
    grouped_net = pm4py.objects.conversion.wf_net.variants.to_process_tree.group_blocks_in_net(net,
                                                                                               parameters=parameters)  # this function finds all block-structures in form of process-tree labels for transitions (one transition corresponds to one block-structure)
    pn_viz.view(pn_viz.apply(grouped_net))

    parallels = {}
    exclusives = {}
    acts = []
    for t in range(len(list(net.transitions))):
        acts.append(list(net.transitions)[t].label)
    for trans in range(len(list(
            grouped_net.transitions))):  # here, we iterate over all labels and collect tree parts with XOR- and AND-operator
        if not list(grouped_net.transitions)[trans].label is None and not str(
                list(grouped_net.transitions)[trans].label) in acts:
            pt_str = list(grouped_net.transitions)[trans].label
            pt = pt_util.parse(pt_str)
            tree_part = pt_util.fold(pt) if fold else pt
            tree_sort(tree_part)
            parallels, exclusives = relations(tree_part, parallels, exclusives)

    solution_patterns = pd.DataFrame(
        index=variants_deviating.index)  # this stores the solution patterns, i.e., which will be the decisions for each variant
    done = 0
    manager_patterns = pd.DataFrame(index=variants_deviating.index)  # this stores the post-processed results

    for var in vars_to_analyze:
        dev_in_var = []
        i = variants_deviating['first_idx'][var]
        for j in range(0, len(aligned_traces[i]['alignment'])):
            if aligned_traces[i]['alignment'][j][1] == None or aligned_traces[i]['alignment'][j][0] == \
                    aligned_traces[i]['alignment'][j][1]:
                next  # we do not care for simultaneous or silent moves
            else:
                if aligned_traces[i]['alignment'][j][1] == '>>':
                    dev_in_var.append(str('log_' + str(j) + '_' + aligned_traces[i]['alignment'][j][0]))
                elif aligned_traces[i]['alignment'][j][0] == '>>':
                    dev_in_var.append(str('model_' + str(j) + '_' + aligned_traces[i]['alignment'][j][1]))

        devs_info = pd.DataFrame(data=None, columns=['activity', 'type', 'alignment_position'],
                                 index=dev_in_var)  # find and store all deviations in the variant
        for j in range(0, len(aligned_traces[i]['alignment'])):
            if aligned_traces[i]['alignment'][j][1] == None or aligned_traces[i]['alignment'][j][0] == \
                    aligned_traces[i]['alignment'][j][1]:
                next  # we do not care for simultaneous or silent moves
            else:
                if aligned_traces[i]['alignment'][j][1] == '>>':
                    devs_info['activity'][str('log_' + str(j) + '_' + aligned_traces[i]['alignment'][j][0])] = \
                        aligned_traces[i]['alignment'][j][0]
                    devs_info['type'][str('log_' + str(j) + '_' + aligned_traces[i]['alignment'][j][0])] = 'log'
                    devs_info['alignment_position'][str('log_' + str(j) + '_' + aligned_traces[i]['alignment'][j][0])] = j
                elif aligned_traces[i]['alignment'][j][0] == '>>':
                    devs_info['activity'][str('model_' + str(j) + '_' + aligned_traces[i]['alignment'][j][1])] = \
                        aligned_traces[i]['alignment'][j][1]
                    devs_info['type'][str('model_' + str(j) + '_' + aligned_traces[i]['alignment'][j][1])] = 'model'
                    devs_info['alignment_position'][str('model_' + str(j) + '_' + aligned_traces[i]['alignment'][j][1])] = j

        swap_patterns = {}
        replace_patterns = {}
        repeated_patterns = {}
        repl = 1
        repea = 1
        swa = 1
        swapped_patterns_single = []
        replace_patterns_single = []
        repeated_patterns_single = []
        # now we extract all instantiations of deviation patterns (this part is referred to as Algorithm 1 in the paper)
        for i, d in enumerate(dev_in_var):
            #swapped
            swapped_block1 = []
            dev_block1 = []
            count = devs_info['activity'].value_counts()[devs_info['activity'][d]]
            typ_1 = devs_info['type'][d]
            if count > 1:
                next_dev_with_activity = False
                swapped_block1.append(devs_info['activity'][d])
                dev_block1.append(d)
                j = i
                while j + 1 < len(dev_in_var):
                    j += 1

                    if typ_1 == devs_info['type'][dev_in_var[j]] and (
                            devs_info['alignment_position'][dev_in_var[j - 1]] == devs_info['alignment_position'][
                        dev_in_var[j]] - 1 or all(
                        aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] == None for run in
                        range(devs_info['alignment_position'][dev_in_var[j - 1]], devs_info['alignment_position'][
                            dev_in_var[j]]))):
                        swapped_block1.append(devs_info['activity'][dev_in_var[j]])
                        dev_block1.append(dev_in_var[j])
                    else:
                        j = 100000000
                j = i

                while next_dev_with_activity == False and j + 1 < len(dev_in_var):
                    j += 1
                    if devs_info['activity'][dev_in_var[i]] == devs_info['activity'][dev_in_var[j]] and not \
                    devs_info['type'][
                        dev_in_var[
                            i]] == \
                    devs_info['type'][
                        dev_in_var[
                            j]]:  # yes if there is log and model move on same activity
                        too_many_block1 = []
                        dev_block2 = [dev_in_var[j]]
                        for check_runner in range(j + 1, j + len(dev_block1)):
                            if check_runner >= len(dev_in_var):
                                next
                            elif devs_info['alignment_position'][dev_in_var[check_runner]] >= len(
                                    aligned_traces[variants_deviating['first_idx'][var]]['alignment']):
                                next
                            elif not devs_info['alignment_position'][dev_in_var[check_runner]] < len(
                                    aligned_traces[variants_deviating['first_idx'][var]]['alignment']):
                                del dev_block1[-(check_runner - j):]
                                del swapped_block1[-(check_runner - j):]
                                break
                            elif not ((aligned_traces[variants_deviating['first_idx'][var]]['alignment'][
                                           devs_info['alignment_position'][dev_in_var[check_runner]]][1] == '>>' and
                                       aligned_traces[variants_deviating['first_idx'][var]]['alignment'][
                                           devs_info['alignment_position'][dev_in_var[check_runner]]][0] == swapped_block1[
                                           check_runner - j]) or (
                                              aligned_traces[variants_deviating['first_idx'][var]]['alignment'][
                                                  devs_info['alignment_position'][dev_in_var[check_runner]]][0] == '>>' and
                                              aligned_traces[variants_deviating['first_idx'][var]]['alignment'][
                                                  devs_info['alignment_position'][dev_in_var[check_runner]]][1] ==
                                              swapped_block1[check_runner - j])):
                                too_many_block1.append(check_runner - j)
                            else:
                                dev_block2.append(dev_in_var[check_runner])
                        for index in range(len(swapped_block1) - 1, -1, -1):
                            if not swapped_block1[index] in ''.join(dev_block2):
                                del swapped_block1[index]
                                del dev_block1[index]
                        next_dev_with_activity = not next_dev_with_activity
                        swapped_activities = []
                        for move in range(devs_info['alignment_position'][dev_in_var[i]] + 1,
                                          devs_info['alignment_position'][dev_in_var[j]]):
                            if not str(aligned_traces[variants_deviating['first_idx'][var]]['alignment'][move][
                                           0]) == '>>' and not str(
                                    aligned_traces[variants_deviating['first_idx'][var]]['alignment'][move][
                                        0]) in swapped_block1:
                                swapped_activities.append(
                                    str(aligned_traces[variants_deviating['first_idx'][var]]['alignment'][move][0]))
                        idexes = []
                        # this annotates deviation patterns
                        for missing in range(len(swapped_activities)):
                            for par in range(len(parallels.keys())):
                                for p in range(len(parallels[list(parallels.keys())[par]])):
                                    if str(parallels[list(parallels.keys())[par]][p]) == str(swapped_activities[missing]):
                                        print(str(parallels[list(parallels.keys())[par]][p]) == str(
                                            swapped_activities[missing]), swapped_activities[missing],
                                              parallels[list(parallels.keys())[par]][p])
                                        idexes.append(missing)
                                print(len(idexes), len(parallels[list(parallels.keys())[par]]))
                                if len(idexes) == len(parallels[list(parallels.keys())[par]]):
                                    print(1)
                                    for i in sorted(idexes, reverse=True):
                                        if i == idexes[0]:
                                            swapped_activities[idexes[0]] = str(
                                                '+(' + str(parallels[list(parallels.keys())[par]]) + ')')
                                        else:
                                            del swapped_activities[i]


                        if len(swapped_block1) > 0 and len(swapped_activities) > 0:
                            if devs_info['type'][dev_in_var[i]] == 'log':
                                if not str('swap_' + ', '.join(sorted(swapped_activities)) + ' and ' + ', '.join(
                                        swapped_block1)) in swapped_patterns_single:
                                    swapped_patterns_single.append(
                                        str('swap_' + ', '.join(sorted(swapped_activities)) + ' and ' + ', '.join(
                                            swapped_block1)))
                                if not str(str(swa) + 'swap_' + ', '.join(sorted(swapped_activities)) + ' and ' + ', '.join(
                                        swapped_block1)) in swap_patterns.keys():
                                    swap_patterns[
                                        str(str(swa) + 'swap_' + ', '.join(
                                            sorted(swapped_activities)) + ' and ' + ', '.join(swapped_block1))] = str(
                                        '-'.join(dev_block1) + '-' + '-'.join(dev_block2))
                                    swa += 1
                            elif devs_info['type'][dev_in_var[i]] == 'model':
                                if not str('swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                        sorted(swapped_activities))) in swapped_patterns_single:
                                    swapped_patterns_single.append(
                                        str('swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                            sorted(swapped_activities))))
                                if not str(str(swa) + 'swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                        sorted(swapped_activities))) in swap_patterns.keys():
                                    swap_patterns[str(str(swa) + 'swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                        sorted(swapped_activities)))] = str(
                                        '-'.join(dev_block1) + '-' + '-'.join(dev_block2))
                                    swa += 1

            act_1 = devs_info['activity'][d]
            for excl in range(len(exclusives.keys())):
                for ex in range(len(exclusives[list(exclusives.keys())[excl]])):
                    if str(exclusives[list(exclusives.keys())[excl]][ex]) == str(act_1):
                        swapped_block1.append(devs_info['activity'][d])
                        dev_block1.append(d)
                        dev_block2 = []
                        j = i
                        while j + 1 < len(dev_in_var):
                            j += 1
                            act_2 = devs_info['activity'][dev_in_var[j]]
                            typ_2 = devs_info['type'][dev_in_var[j]]
                            for excl9 in range(len(exclusives.keys())):
                                for ex9 in range(len(exclusives[list(exclusives.keys())[excl9]])):
                                    if str(exclusives[list(exclusives.keys())[excl9]][ex9]) == str(
                                            act_2) and typ_2 == typ_1 and not dev_in_var[j] in dev_block1:
                                        dev_block1.append(dev_in_var[j])
                                        swapped_block1.append(devs_info['activity'][dev_in_var[j]])
                            for ex2 in range(len(exclusives[list(exclusives.keys())[excl]])):
                                if str(exclusives[list(exclusives.keys())[excl]][ex2]) == str(act_2) and typ_2 != typ_1:
                                    dev_block2.append(dev_in_var[j])
                                    swapped_activities = []
                                    for move in range(devs_info['alignment_position'][dev_in_var[i]] + 1,
                                                      devs_info['alignment_position'][dev_in_var[j]]):
                                        if not str(aligned_traces[variants_deviating['first_idx'][var]]['alignment'][move][
                                                       0]) == '>>' and not str(
                                                aligned_traces[variants_deviating['first_idx'][var]]['alignment'][move][
                                                    0]) in swapped_block1:
                                            swapped_activities.append(
                                                str(aligned_traces[variants_deviating['first_idx'][var]]['alignment'][move][
                                                        0]))

                                    if len(swapped_block1) > 0 and len(swapped_activities) > 0:
                                        if devs_info['type'][dev_in_var[i]] == 'log':
                                            if not str(
                                                    'swap_' + ', '.join(sorted(swapped_activities)) + ' and ' + ', '.join(
                                                            swapped_block1)) in swapped_patterns_single:
                                                swapped_patterns_single.append(str('swap_' + ', '.join(
                                                    sorted(swapped_activities)) + ' and ' + ', '.join(swapped_block1)))
                                            if not str(str(swa) + 'swap_' + ', '.join(
                                                    sorted(swapped_activities)) + ' and ' + ', '.join(
                                                    swapped_block1)) in swap_patterns.keys():
                                                swap_patterns[
                                                    str(str(swa) + 'swap_' + ', '.join(
                                                        sorted(swapped_activities)) + ' and ' + ', '.join(
                                                        swapped_block1))] = str(
                                                    '-'.join(dev_block1) + '-' + '-'.join(dev_block2))
                                                swa += 1
                                        elif devs_info['type'][dev_in_var[i]] == 'model':
                                            if not str('swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                                    sorted(swapped_activities))) in swapped_patterns_single:
                                                swapped_patterns_single.append(
                                                    str('swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                                        sorted(swapped_activities))))
                                            if not str(str(swa) + 'swap_' + ', '.join(swapped_block1) + ' and ' + ', '.join(
                                                    sorted(swapped_activities))) in swap_patterns.keys():
                                                swap_patterns[str(str(swa) + 'swap_' + ', '.join(
                                                    swapped_block1) + ' and ' + ', '.join(
                                                    sorted(swapped_activities)))] = str(
                                                    '-'.join(dev_block1) + '-' + '-'.join(dev_block2))
                                                swa += 1
            #replaced
            if len(devs_info) - i > 2 and (
                    devs_info['alignment_position'][dev_in_var[i]] == devs_info['alignment_position'][
                dev_in_var[i + 1]] - 1 or all(
                aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] == None for run in
                range(devs_info['alignment_position'][dev_in_var[i]], devs_info['alignment_position'][
                    dev_in_var[i + 1]]))):
                consequetive_devs = 2
                multi_log = []
                multi_model = []
                multi_log_devs = []
                multi_model_devs = []
                for runner in range(i + 2, len(devs_info)):
                    if (devs_info['alignment_position'][dev_in_var[runner - 1]] == devs_info['alignment_position'][
                        dev_in_var[runner]] - 1 or all(
                        aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] == None for run in
                        range(devs_info['alignment_position'][dev_in_var[runner - 1]], devs_info['alignment_position'][
                            dev_in_var[runner]]))):
                        consequetive_devs += 1
                    else:
                        break
                for z in range(i, i + consequetive_devs):
                    if devs_info['type'][dev_in_var[z]] == 'model':
                        multi_model.append(devs_info['activity'][dev_in_var[z]])
                        multi_model_devs.append(str(dev_in_var[z]))
                    elif devs_info['type'][dev_in_var[z]] == 'log':
                        multi_log.append(devs_info['activity'][dev_in_var[z]])
                        multi_log_devs.append(str(dev_in_var[z]))

                idexes = []
                for missing in range(len(multi_model)):
                    for excl in range(len(exclusives.keys())):
                        for ex in range(len(exclusives[list(exclusives.keys())[excl]])):
                            if str(exclusives[list(exclusives.keys())[excl]][ex]) == str(multi_model[missing]):
                                print(str(exclusives[list(exclusives.keys())[excl]][ex]) == str(multi_model[missing]),
                                      multi_model[missing], exclusives[list(exclusives.keys())[excl]][ex])
                                multi_model[missing] = str('x(' + str(exclusives[list(exclusives.keys())[excl]]) + ')')
                    for par in range(len(parallels.keys())):
                        for p in range(len(parallels[list(parallels.keys())[par]])):
                            if str(parallels[list(parallels.keys())[par]][p]) == str(multi_model[missing]):
                                print(str(parallels[list(parallels.keys())[par]][p]) == str(multi_model[missing]),
                                      multi_model[missing], parallels[list(parallels.keys())[par]][p])
                                idexes.append(missing)
                        print(len(idexes), len(parallels[list(parallels.keys())[par]]))
                        if len(idexes) == len(parallels[list(parallels.keys())[par]]):
                            print(1)
                            for i in sorted(idexes, reverse=True):
                                if i == idexes[0]:
                                    multi_model[idexes[0]] = str('+(' + str(parallels[list(parallels.keys())[par]]) + ')')
                                else:
                                    del multi_model[i]

                if not (len(multi_log) == 0 or len(multi_model) == 0):
                    if not str(
                            'replace_' + ', '.join(multi_model) + ' by ' + ', '.join(multi_log)) in replace_patterns_single:
                        replace_patterns_single.append(
                            str('replace_' + ', '.join(multi_model) + ' by ' + ', '.join(multi_log)))
                    replace_patterns[
                        str(str(repl) + 'replace_' + ', '.join(multi_model) + ' by ' + ', '.join(multi_log))] = str(
                        '-'.join(multi_model_devs) + '_' + '-'.join(multi_log_devs))
                    repl = +1
            elif len(devs_info) - i > 1 and (
                    devs_info['alignment_position'][dev_in_var[i]] == devs_info['alignment_position'][
                dev_in_var[i + 1]] - 1 or all(
                aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] == None for run in
                range(devs_info['alignment_position'][dev_in_var[i]], devs_info['alignment_position'][
                    dev_in_var[i + 1]]))) and not devs_info['type'][dev_in_var[i]] == devs_info['type'][dev_in_var[i + 1]]:
                j = i + 1
                possible_multi_type = devs_info['type'][dev_in_var[j]]
                possible_multi_acts = [devs_info['activity'][dev_in_var[j]]]
                multi_devs = [str(dev_in_var[j])]
                while j < len(devs_info) - 1:
                    if devs_info['type'][dev_in_var[j]] == devs_info['type'][dev_in_var[j + 1]] and (
                            devs_info['alignment_position'][dev_in_var[j]] == devs_info['alignment_position'][
                        dev_in_var[j + 1]] - 1 or all(
                        aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] == None for run in
                        range(devs_info['alignment_position'][dev_in_var[j]], devs_info['alignment_position'][
                            dev_in_var[j + 1]]))):
                        possible_multi_acts.append(devs_info['activity'][dev_in_var[j + 1]])
                        multi_devs.append(str(dev_in_var[j + 1]))
                        j += 1
                    else:
                        j = 100000
                if len(multi_devs) <= 1:
                    if devs_info['type'][dev_in_var[i]] == 'log':
                        log_act = devs_info['activity'][dev_in_var[i]]
                        model_act = devs_info['activity'][dev_in_var[i + 1]]
                    elif devs_info['type'][dev_in_var[i]] == 'model':
                        log_act = devs_info['activity'][dev_in_var[i + 1]]
                        model_act = devs_info['activity'][dev_in_var[i]]
                    if not str('replace_' + model_act + ' by ' + log_act) in replace_patterns_single:
                        replace_patterns_single.append(str('replace_' + model_act + ' by ' + log_act))
                    replace_patterns[str(str(repl) + 'replace_' + model_act + ' by ' + log_act)] = str(
                        dev_in_var[i] + '-' + dev_in_var[i + 1])
                    repl = +1
                else:
                    if devs_info['type'][dev_in_var[i]] == 'log':
                        log_act = devs_info['activity'][dev_in_var[i]]

                        idexes = []
                        for missing in range(len(possible_multi_acts)):
                            for excl in range(len(exclusives.keys())):
                                for ex in range(len(exclusives[list(exclusives.keys())[excl]])):
                                    if str(exclusives[list(exclusives.keys())[excl]][ex]) == str(
                                            possible_multi_acts[missing]):
                                        print(str(exclusives[list(exclusives.keys())[excl]][ex]) == str(
                                            possible_multi_acts[missing]), possible_multi_acts[missing],
                                              exclusives[list(exclusives.keys())[excl]][ex])
                                        possible_multi_acts[missing] = str(
                                            'x(' + str(exclusives[list(exclusives.keys())[excl]]) + ')')
                            for par in range(len(parallels.keys())):
                                for p in range(len(parallels[list(parallels.keys())[par]])):
                                    if str(parallels[list(parallels.keys())[par]][p]) == str(possible_multi_acts[missing]):
                                        print(str(parallels[list(parallels.keys())[par]][p]) == str(
                                            possible_multi_acts[missing]), possible_multi_acts[missing],
                                              parallels[list(parallels.keys())[par]][p])
                                        idexes.append(missing)
                                print(len(idexes), len(parallels[list(parallels.keys())[par]]))
                                if len(idexes) == len(parallels[list(parallels.keys())[par]]):
                                    print(1)
                                    for i in sorted(idexes, reverse=True):
                                        if i == idexes[0]:
                                            possible_multi_acts[idexes[0]] = str(
                                                '+(' + str(parallels[list(parallels.keys())[par]]) + ')')
                                        else:
                                            del possible_multi_acts[i]
                        model_act = ', '.join(possible_multi_acts)
                    elif devs_info['type'][dev_in_var[i]] == 'model':
                        log_act = ', '.join(possible_multi_acts)
                        model_act = devs_info['activity'][dev_in_var[i]]
                    if not str('replace_' + model_act + ' by ' + log_act) in replace_patterns_single:
                        replace_patterns_single.append(str('replace_' + model_act + ' by ' + log_act))
                    replace_patterns[str(str(repl) + 'replace_' + model_act + ' by ' + log_act)] = str(
                        dev_in_var[i] + '-' + '-'.join(multi_devs))
                    repl = +1

            #repeated
            if devs_info['type'][dev_in_var[i]] == 'log' and devs_info['activity'][dev_in_var[i]] in list(
                    aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][0] for run in
                    range(devs_info['alignment_position'][dev_in_var[i]])) and devs_info['activity'][dev_in_var[i]] in list(
                aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] for run in
                range(devs_info['alignment_position'][dev_in_var[i]])):
                if len(devs_info) - i >= 1:
                    potential_multi = [devs_info['activity'][dev_in_var[i]]]
                    multi_devs = [str(dev_in_var[i])]
                    for multi in range(i + 1, len(devs_info)):
                        if not devs_info['type'][dev_in_var[multi]] == 'log' or not \
                                aligned_traces[variants_deviating['first_idx'][var]]['alignment'][
                                    devs_info['alignment_position'][dev_in_var[multi]]][1] == '>>' or not \
                        devs_info['activity'][
                            dev_in_var[
                                multi]] in list(
                            aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][0] for run in
                            range(devs_info['alignment_position'][dev_in_var[multi]])) or not (
                                devs_info['alignment_position'][dev_in_var[multi - 1]] == devs_info['alignment_position'][
                            dev_in_var[multi]] - 1 or all(
                            aligned_traces[variants_deviating['first_idx'][var]]['alignment'][run][1] == None for run in
                            range(devs_info['alignment_position'][dev_in_var[multi - 1]], devs_info['alignment_position'][
                                dev_in_var[multi]]))):
                            break
                        else:
                            potential_multi.append(devs_info['activity'][dev_in_var[multi]])
                            multi_devs.append(str(dev_in_var[multi]))
                            multi_length = multi - i
                    if not str('repeated_' + ' and '.join(potential_multi)) in repeated_patterns_single:
                        repeated_patterns_single.append(str('repeated_' + ' and '.join(potential_multi)))
                    repeated_patterns[str(str(repea) + 'repeated_' + ' and '.join(potential_multi))] = '-'.join(multi_devs)

                    repea += 1

        for key in swap_patterns.keys():
            if not key in solution_patterns.columns:
                solution_patterns[key] = 0
        for key in replace_patterns.keys():
            if not key in solution_patterns.columns:
                solution_patterns[key] = 0
        for key in repeated_patterns.keys():
            if not key in solution_patterns.columns:
                solution_patterns[key] = 0
        for key in dev_in_var:
            if not key in solution_patterns.columns:
                solution_patterns[key] = 0

        # from here on, we build the linear program
        model = Model(name="solution_variants")
        indiv_devs = model.binary_var_list(dev_in_var)
        swap_pattern_devs = model.binary_var_list(list(swap_patterns.keys()))
        replace_pattern_devs = model.binary_var_list(list(replace_patterns.keys()))
        repeated_pattern_devs = model.binary_var_list(list(repeated_patterns.keys()))
        model.add_constraints(model.sum(indiv_devs[i]) + model.sum(
            ((indiv_devs[i].name in swap_patterns[swap_pattern_devs[j].name]) * 1 * swap_pattern_devs[j]) for j in
            range(len(swap_patterns.keys()))) + model.sum(
            ((indiv_devs[i].name in replace_patterns[replace_pattern_devs[j].name]) * 1 * replace_pattern_devs[j]) for j in
            range(len(replace_patterns.keys()))) + model.sum(
            ((indiv_devs[i].name in repeated_patterns[repeated_pattern_devs[j].name]) * 1 * repeated_pattern_devs[j]) for j
            in
            range(len(repeated_patterns.keys()))) == 1 for i in range(len(dev_in_var)))
        objective = model.sum(indiv_devs[i] for i in range(len(dev_in_var))) * bigM + model.sum(
            swap_pattern_devs[j] for j in range(len(swap_pattern_devs))) * cost_swap + model.sum(
            replace_pattern_devs[j] for j in range(len(replace_pattern_devs))) * cost_replace + model.sum(
            repeated_pattern_devs[j] for j in range(len(repeated_pattern_devs))) * cost_repeat
        model.minimize(objective)
        model.solve()
        model.solve_details
        for j in indiv_devs:
            solution_patterns[j.name][var] = j.solution_value
        for j in swap_pattern_devs:
            solution_patterns[j.name][var] = j.solution_value
        for j in replace_pattern_devs:
            solution_patterns[j.name][var] = j.solution_value
        for j in repeated_pattern_devs:
            solution_patterns[j.name][var] = j.solution_value
        done += 1

        # post-processing starts here
        solution_patterns['deviating'] = variants_deviating['deviating']
        manager_patterns['deviating'] = solution_patterns['deviating']
        manager_patterns['count'] = variants_deviating['count']
        manager_patterns['label'] = variants_deviating['label']

        for single in swapped_patterns_single:
            for key in list(swap_patterns.keys()):
                if single in key and len(single) == len(key) - 1:
                    if not single in manager_patterns.keys():
                        manager_patterns[single] = 0
                    manager_patterns[single][var] += solution_patterns[key][var]
        for single in replace_patterns_single:
            for key in list(replace_patterns.keys()):
                if single in key and len(single) == len(key) - 1:
                    if not single in manager_patterns.keys():
                        manager_patterns[single] = 0
                    manager_patterns[single][var] += solution_patterns[key][var]
        for single in repeated_patterns_single:
            for key in list(repeated_patterns.keys()):
                if single in key and len(single) == len(key) - 1:
                    if not single in manager_patterns.keys():
                        manager_patterns[single] = 0
                    manager_patterns[single][var] += solution_patterns[key][var]
        used_moves = 0

        for i, dev_idx in enumerate(list(devs_info.index)):

            if len(devs_info) - i > 1:
                if solution_patterns[dev_idx][var] == 1 and devs_info['alignment_position'][dev_idx] > used_moves and \
                        devs_info['alignment_position'][list(devs_info.index)[i]] == devs_info['alignment_position'][
                    list(devs_info.index)[i + 1]] - 1:
                    list_activities = [str(devs_info['activity'][dev_idx])]
                    j = i + 1
                    if j < len(devs_info):
                        while j <= len(list(devs_info.index)) - 1:
                            if solution_patterns[list(devs_info.index)[j]][var] == 1 and devs_info['type'][dev_idx] == \
                                    devs_info['type'][list(devs_info.index)[j]] and devs_info['alignment_position'][
                                list(devs_info.index)[j]] == devs_info['alignment_position'][
                                list(devs_info.index)[j - 1]] + 1:
                                list_activities.append(str(devs_info['activity'][list(devs_info.index)[j]]))
                                used_moves = devs_info['alignment_position'][list(devs_info.index)[j]]
                            j += 1
                    if devs_info['type'][dev_idx] == 'log':
                        type = 'inserted_'
                    else:
                        type = 'missing_'
                    if type == 'missing_':
                        idexes = []
                        for missing in range(len(list_activities)):
                            for excl in range(len(exclusives.keys())):
                                for ex in range(len(exclusives[list(exclusives.keys())[excl]])):
                                    if str(exclusives[list(exclusives.keys())[excl]][ex]) == str(list_activities[missing]):
                                        print(str(exclusives[list(exclusives.keys())[excl]][ex]) == str(
                                            list_activities[missing]), list_activities[missing],
                                              exclusives[list(exclusives.keys())[excl]][ex])
                                        list_activities[missing] = str(
                                            'x(' + str(exclusives[list(exclusives.keys())[excl]]) + ')')
                            for par in range(len(parallels.keys())):
                                for p in range(len(parallels[list(parallels.keys())[par]])):
                                    if str(parallels[list(parallels.keys())[par]][p]) == str(list_activities[missing]):
                                        print(
                                            str(parallels[list(parallels.keys())[par]][p]) == str(list_activities[missing]),
                                            list_activities[missing], parallels[list(parallels.keys())[par]][p])
                                        idexes.append(missing)
                                print(len(idexes), len(parallels[list(parallels.keys())[par]]))
                                if len(idexes) == len(parallels[list(parallels.keys())[par]]):
                                    print(1)
                                    for i in sorted(idexes, reverse=True):
                                        if i == idexes[0]:
                                            list_activities[idexes[0]] = str(
                                                '+(' + str(parallels[list(parallels.keys())[par]]) + ')')
                                        else:
                                            del list_activities[i]
                    print(1, i, list_activities)
                    cluster = str(type + ', '.join(list_activities))
                    if not cluster in manager_patterns.columns:
                        manager_patterns[cluster] = 0
                    manager_patterns[cluster][var] += 1
                elif solution_patterns[dev_idx][var] == 1 and devs_info['alignment_position'][dev_idx] > used_moves:
                    list_activities = [str(devs_info['activity'][dev_idx])]
                    if devs_info['type'][dev_idx] == 'log':
                        type = 'inserted_'
                    else:
                        type = 'missing_'
                    if type == 'missing_':
                        idexes = []
                        for missing in range(len(list_activities)):
                            for excl in range(len(exclusives.keys())):
                                for ex in range(len(exclusives[list(exclusives.keys())[excl]])):
                                    if str(exclusives[list(exclusives.keys())[excl]][ex]) == str(list_activities[missing]):
                                        print(str(exclusives[list(exclusives.keys())[excl]][ex]) == str(
                                            list_activities[missing]), list_activities[missing],
                                              exclusives[list(exclusives.keys())[excl]][ex])
                                        list_activities[missing] = str(
                                            'x(' + str(exclusives[list(exclusives.keys())[excl]]) + ')')
                            for par in range(len(parallels.keys())):
                                for p in range(len(parallels[list(parallels.keys())[par]])):
                                    if str(parallels[list(parallels.keys())[par]][p]) == str(list_activities[missing]):
                                        print(
                                            str(parallels[list(parallels.keys())[par]][p]) == str(list_activities[missing]),
                                            list_activities[missing], parallels[list(parallels.keys())[par]][p])
                                        idexes.append(missing)
                                print(len(idexes), len(parallels[list(parallels.keys())[par]]))
                                if len(idexes) == len(parallels[list(parallels.keys())[par]]):
                                    print(1)
                                    for i in sorted(idexes, reverse=True):
                                        if i == idexes[0]:
                                            list_activities[idexes[0]] = str(
                                                '+(' + str(parallels[list(parallels.keys())[par]]) + ')')
                                        else:
                                            del list_activities[i]
                    print(2, list_activities)
                    cluster = str(type + ', '.join(list_activities))
                    if not cluster in manager_patterns.columns:
                        manager_patterns[cluster] = 0
                    manager_patterns[cluster][var] += 1
                    used_moves = devs_info['alignment_position'][list(devs_info.index)[i]]
            elif solution_patterns[dev_idx][var] == 1 and (
                    devs_info['alignment_position'][dev_idx] > used_moves or devs_info['alignment_position'][
                dev_idx] == 0 or len(devs_info) == 1):
                list_activities = [str(devs_info['activity'][dev_idx])]
                if devs_info['type'][dev_idx] == 'log':
                    type = 'inserted_'
                else:
                    type = 'missing_'
                if type == 'missing_':
                    idexes = []
                    for missing in range(len(list_activities)):
                        for excl in range(len(exclusives.keys())):
                            for ex in range(len(exclusives[list(exclusives.keys())[excl]])):
                                if str(exclusives[list(exclusives.keys())[excl]][ex]) == str(list_activities[missing]):
                                    print(
                                        str(exclusives[list(exclusives.keys())[excl]][ex]) == str(list_activities[missing]),
                                        list_activities[missing], exclusives[list(exclusives.keys())[excl]][ex])
                                    list_activities[missing] = str(
                                        'x(' + str(exclusives[list(exclusives.keys())[excl]]) + ')')
                        for par in range(len(parallels.keys())):
                            for p in range(len(parallels[list(parallels.keys())[par]])):
                                if str(parallels[list(parallels.keys())[par]][p]) == str(list_activities[missing]):
                                    print(str(parallels[list(parallels.keys())[par]][p]) == str(list_activities[missing]),
                                          list_activities[missing], parallels[list(parallels.keys())[par]][p])
                                    idexes.append(missing)
                            print(len(idexes), len(parallels[list(parallels.keys())[par]]))
                            if len(idexes) == len(parallels[list(parallels.keys())[par]]):
                                print(1)
                                for i in sorted(idexes, reverse=True):
                                    if i == idexes[0]:
                                        list_activities[idexes[0]] = str(
                                            '+(' + str(parallels[list(parallels.keys())[par]]) + ')')
                                    else:
                                        del list_activities[i]
                cluster = str(type + ', '.join(list_activities))
                if not cluster in manager_patterns.columns:
                    manager_patterns[cluster] = 0
                manager_patterns[cluster][var] += 1
                used_moves = devs_info['alignment_position'][list(devs_info.index)[i]]
    solution_patterns = solution_patterns.sort_index(axis=1).drop(['deviating'], axis=1)

    path = (os.getcwd() + '/Output')
    timer = pd.DataFrame(data=0, columns=['time'], index=[0])
    timer['time'][0] = time.clock() - time_start

    xt, z = os.path.split(file)
    writer = pd.ExcelWriter(
        path + '/wo_cf_new_' + z + '.xlsx',
        engine="xlsxwriter")

    timer.to_excel(writer, sheet_name=('Time'))
    export_solutions = pd.DataFrame(index=solution_patterns.index, columns=['deviating', 'count', 'label'])
    export_solutions['deviating'] = manager_patterns['deviating']
    export_solutions['count'] = manager_patterns['count']
    export_solutions['label'] = manager_patterns['label']
    export_solutions
    manager_patterns = manager_patterns.sort_index(axis=1, ascending=False)
    mgt_pts = manager_patterns.drop(columns=['deviating', 'count', 'label'])
    for i, index in enumerate(list(export_solutions.index)):
        counter = 0
        for col in mgt_pts.columns:
            if not manager_patterns['label'][index] == '' and counter > 0:
                continue
            if mgt_pts[col][index] > 0:
                if not str('pld_' + str(counter)) in export_solutions.columns:
                    export_solutions[str('pld_' + str(counter))] = 0
                if not str('count_pld_' + str(counter)) in export_solutions.columns:
                    export_solutions[str('count_pld_' + str(counter))] = 0
                export_solutions[str('pld_' + str(counter))][index] = col
                export_solutions[str('count_pld_' + str(counter))][index] += mgt_pts[col][index]
                counter += 1
    export_solutions.to_excel(writer, sheet_name=('export'))
    writer.close()
    return manager_patterns, variants_deviating, solution_patterns

In [ ]:
manager_patterns, variants_deviating, solution_patterns=discover_patterns_wo_cf(log, net, initial_marking, final_marking)

In [ ]:
x